## 2)  Creación de dataset prediccion

En esta etapa generamos las imágenes que utilizará el modelo para el entrenamiento y la predicción. 

Utilizando las imágenes generadas en el paso anterior y la información vectorial de la ubicación de los asentamientos, generamos el dataset utilizado luego por el modelo de ML. Este dataset se genera con el formato necesario para el modelo.


### Instalacion de pysatproc (correr solo una vez)

In [ ]:
#!pip install -U pysatproc

### Variables para configuracion de satproc_extract_chips
Definimos el la configuracion para la predicción. Debemos pasar la ruta a las imagenes del dataset de prediccion y al modelo.

In [ ]:

path_to_files = "../images-S2/zona_4/2021-01-01_2021-03-01/*.tif"             # ruta a las imagenes satelitales
output_folder = "../dataset/data_predict/zona_4/v2/160_80/"                   # ruta donde se van a dejar las imagenes porcesadas
vector_aoi_file_path = "../data/shp/predict_aoi_zona4_buff5km_4326.geojson"   # archivo geojson con el area de interes                       
size = 160                                                                    # stamaño (pixeles) de las imagenes que se van a generar
step_size = size                                                              # tamaño de la ventana

### Generación del dataset para predecir

In [ ]:
#Creo dataset para prediccion (solo imagenes)
!satproc_extract_chips $path_to_files \
    -o $output_folder \
    --aoi $vector_aoi_file_path \
    --size $size \
    --step-size $step_size \
    --rescale \
    --rescale-mode s2_rgb_extra --lower-cut 1 --upper-cut 100


# 3) Prediccion

### Librerias
Descargamos la librería os para la navegación de archivos, y unetseg para la red neuronal

In [ ]:
#importo librerias
from unetseg.predict import PredictConfig, predict
from unetseg.evaluate import plot_data_results
import os

### Variables para configuracion de la predicción

In [ ]:
zona='zona_4'
version = 'v2'
size_stepsize='160_80'

predict_config = PredictConfig(
                                    images_path = os.path.join('../dataset/data_predict',zona,version,size_stepsize),
                                    results_path = os.path.join('../dataset/data_results',zona,version,size_stepsize),
                                    batch_size=16,
                                    model_path='../data/weights/model/zona_3UNet_techo_4D_spe100_img160_size160_sz80.h5',  #  ruta al modelo (.h5)
                                    height=160,
                                    width=160,
                                    n_channels=4,
                                    n_classes=1,
                                    class_weights=[1])

### Corro la prediccion

In [ ]:
predict(predict_config)

Podemos ver algunos resultados preliminales de la prediccion

## 4) Post-procesamiento
Post-procesamiento que fueron aplicados sobre los resultados de las predicciones del modelo de U-Net.

### Librerias

In [ ]:
from satproc.postprocess.polygonize import polygonize 
from satproc.filter import filter_by_max_prob

### Filtramos las predicciones con probabilidad mayor al parametro y poligonos mayor a una superficie determinada

Aplica una rutina de poligonización sobre los resultados de la predicción del modelo y genera un archivo vectorial en formato GeoPackage (GPKG). La rutina aplica `gdal_polygonize.py` a cada chip resultado generando un GPKG para cada chip, y luego une todos estos archivos en uno solo, de manera eficiente.

Antes de unirlos también aplica un umbral sobre los valores de los rásteres, que en este caso representan la probabilidad (valores entre 0 y 1).

Como último paso, filtramos aquellos polígonos de area inferior a 500 m², dado que consideramos que son falsos positivos. Para esto utilizamos `ogr2ogr` y una consulta SQL. El resultado se guarda en `dataset/data_results/`

In [ ]:
parametro_prob=0.3 # parametro a filtrar
min_area = 500     # superficie minima a fitrar

# creamos una carpeta
carpeta = os.path.join('../dataset/data_results',zona,version,size_stepsize)+'_filtered_up'+str(parametro_prob).split('.')[0]+str(parametro_prob).split('.')[1]+'/'
!mkdir -p $carpeta

# filtramos por la probabilidad "parametro_prob"
filter_by_max_prob(input_dir = os.path.join('../dataset/data_results',zona,version,size_stepsize),
                   output_dir = os.path.join('../dataset/data_results',zona,version,size_stepsize)+'_filtered_up'+str(parametro_prob).split('.')[0]+str(parametro_prob).split('.')[1]+'/',
                   threshold = parametro_prob) #Probar ≠ prob umbrales, por ahi, empezar con 0.1


# generamos los poligonos
input_path =  os.path.join('../dataset/data_results',zona,version,size_stepsize)+'_filtered_up'+str(parametro_prob).split('.')[0]+str(parametro_prob).split('.')[1]+'/'
output_path = os.path.join('../dataset/data_results',zona,version+'_'+size_stepsize+'_'+zona+'_thr_'+str(parametro_prob).split('.')[0]+str(parametro_prob).split('.')[1]+'.gpkg')
polygonize(threshold=parametro_prob,
          # value=100,
           input_dir=input_path,
           output=output_path)

# pasamos a proyeccion utm para poder usar metros cuadrados como medida de superficie
src_file = os.path.join('../dataset/data_results',zona,version+'_'+size_stepsize+'_'+zona+'_thr_'+str(parametro_prob).split('.')[0]+str(parametro_prob).split('.')[1]+'.gpkg')
dst_file = os.path.join('../dataset/data_results',zona,version+'_'+size_stepsize+'_'+zona+'_thr_'+str(parametro_prob).split('.')[0]+str(parametro_prob).split('.')[1]+'_utm.gpkg')
!ogr2ogr -s_srs EPSG:4326 -t_srs EPSG:32720 -f 'ESRI Shapefile' $dst_file $src_file #lo puedo abrir en QGIS como archivo vectorial
        
        
# filtramos los que sean mas chico que el parametro "min_area"
input_path = os.path.join('../dataset/data_results',zona,version+'_'+size_stepsize+'_'+zona+'_thr_'+str(parametro_prob).split('.')[0]+str(parametro_prob).split('.')[1]+'_utm.gpkg')
output_path = input_path.replace(".gpkg", "_up"+str(min_area)+".gpkg")    
tabla = input_path.split("/",-1)[-1].strip('.gpkg').strip('_utm')
sql = "SELECT * FROM "+tabla+" m WHERE ST_Area(geometry) > "+str(min_area)        

!ogr2ogr -t_srs EPSG:32720 -f "GPKG" -sql "$sql" -dialect SQLITE -nln results $output_path $input_path

print('Archivo generado correctamente',output_path)